In [1]:
from source.class_yolo_trainer import YOLOTrainer

yolo_trainer = YOLOTrainer
yolo_trainer = YOLOTrainer()

In [2]:
### vou deixar anotado porque não sei se fiz gambiarra, 
""" no atributo image_folder para tarefas de classificação eu estou passando somente a pasta do dataset, sem indicar a pasta de imagens
"""
yolo_trainer.image_folder = r"emissoes_dataset_YOLO\images"
yolo_trainer.annotations_folder = r"emissoes_dataset_YOLO\labels"
yolo_trainer.yolo_Classes = ["emissao"]
yolo_trainer.dataset_path = "emissoes_YOLO"
yolo_trainer.task = "detect"
yolo_trainer.aug = True
yolo_trainer.n_aug = 10
yolo_trainer.odd = 0.12
yolo_trainer.test_percentual_divisor = 20

yolo_trainer.slicing()

In [ ]:
from source.modules.yolo_dataloader import YOLO_Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

yolo_dataloader = YOLO_Dataset
yolo_dataloader = YOLO_Dataset()

train_images_dir = 'path/to/train/labels'
val_images_dir = 'path/to/train/labels'
train_labels_dir = 'path/to/train/labels'
val_labels_dir = 'path/to/train/labels'
batch_size = 30

transform = transforms.Compose([    # Utilizando o novo Dataset no lugar do ImageFolder:
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

train_dataset = YOLO_Dataset(train_images_dir, train_labels_dir, transform=transform)
val_dataset = YOLO_Dataset(val_images_dir, val_labels_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
from ultralytics import YOLO
import yaml

#### YOLO é gambiarra e eu posso provar:
""" Para treinamentos de classificação com YOLO, você deve indicar o dir com o dataset
que deve estar especificado dentro de uma pasta chamada 'datasets'. No entando, para detecção
o YOLO é diferente. Você precisa indicar o caminho do arquivo 'dataset.yaml' para que ele 
possa encontrar o dataset e realizar o treinamento. É a mesma função, de uma mesma lib,
mas os caras fizeram de forma que o mesmo argumento recebe duas entradas completamente 
diferentes a depender do treinamento que você vai fazer.
"""

# Carregar configurações de um arquivo YAML
with open('../hyper_yolo.yaml', 'r') as file:
    config = yaml.safe_load(file)

train_config = config['train']
aug_config = config['train']['augmentation']

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

results = model.train(
    data = "datasets/emissoes_YOLO/dataset.yaml",
    device = "cuda",

    ### training configs. Checkout hyper_yolo.yaml for details
    imgsz = train_config['imgsz'],
    batch = train_config['batch'],
    weight_decay = train_config['weight_decay'],

    warmup_epochs = train_config['warmup_epochs'],
    warmup_momentum = train_config['warmup_momentum'],
    warmup_bias_lr = train_config['warmup_bias_lr'],

    epochs = train_config['epochs'],
    momentum = train_config['momentum'],
    lr0 = train_config['lr0'],
    lrf = train_config['lrf'],
    optimizer = train_config['optimizer'],

    ### if you r willing to use yolo aug, uncomment section bellow
    ### not recomended if ur data is already aug though
    ### augment args
    # hsv_h = aug_config['hsv_h'],
    # hsv_s = aug_config['hsv_s'],
    # hsv_v = aug_config['hsv_v'],
    # degrees = aug_config['degrees'],
    # translate = aug_config['translate'],
    # scale = aug_config['scale'],
    # shear = aug_config['shear'],
    # perspective = aug_config['perspective'],
    # flipud = aug_config['flipud'],
    # fliplr = aug_config['fliplr'],
    # mosaic = aug_config['mosaic'],
    # mixup = aug_config['mixup'],
    # copy_paste = aug_config['copy_paste'],
    # auto_augment = aug_config['auto_augment'],
)

In [ ]:
from ultralytics import YOLO

model = YOLO("runs\\detect\\train52\\weights\\best.pt")

model.info()

In [ ]:
from ultralytics import YOLO

train = 'train52'
model = YOLO(f"runs/detect/{train}/weights/best.pt")
model.predict("../RUIM 23-06-22 (1).mp4", save=True, conf=0.4, device="cuda", save_txt=False, save_conf=True, save_crop=False)

In [ ]:
from utils.check_file import check_file

check_file("datasets\\emissoes_YOLO\\dataset.yaml")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("../emissoes_dataset_CVAT", "emissoes_dataset_YOLO")

In [ ]:
from utils.remover_labels_vazios import processar_dataset

processar_dataset("emissoes_dataset_YOLO")